In [1]:
import numpy as np
import tensorflow as tf
import gym
from gym.envs.registration import register
from keras.layers import Dense
from keras.models import Sequential
from keras import backend as K
from keras.optimizers import Adam
import random

Using TensorFlow backend.


In [15]:
'''
환경 생성
'''
env = gym.make('CartPole-v1')

In [16]:
size_in = env.observation_space.shape[0]
size_out = env.action_space.n
size_w1 = 8
size_w2 = 12
size_w3 = 6
save_file = 'C:\\Users\\김민수\\Documents\\GitHub\\RL\\vars\\cartpole_pg'

lr = .02 ## learning rate
total_episode = 1000
epsilon = .05
gamma = .95  ##Discounted factor

reward = tf.placeholder(tf.float32)
STATE_IN = tf.placeholder(tf.float32, [None, size_in])
W_1 = tf.Variable(tf.random_normal([size_in, size_w1],stddev=.01), name='W_1')
W_2 = tf.Variable(tf.random_normal([size_w1, size_w2],stddev=.01), name='W_2')
W_3 = tf.Variable(tf.random_normal([size_w2, size_w3],stddev=.01), name='W_3')
out = tf.Variable(tf.random_normal([size_w3, size_out],stddev=.01), name='out')

In [17]:
L_1 = tf.matmul(STATE_IN, W_1)
L_2 = tf.matmul(L_1, W_2)
L_3 = tf.matmul(L_2, W_3)
L_out = tf.sigmoid(tf.matmul(L_3, out))

loss = -tf.reduce_mean(tf.log(L_out)*reward) ##pg는 신경망을 거치면 정책(pi)이 바로 나온다.
train = tf.train.AdamOptimizer(lr).minimize(loss)

In [19]:
def discounted_reward(r):
    dr = np.zeros_like(r)
    sum_r = 0
    
    for i in reversed(range(0, r.size)):
        sum_r = gamma*sum_r + r[i]
        dr[i] = sum_r  
    
    return dr

In [20]:
saver = tf.train.Saver()

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    state_buffer = []
    reward_buffer = []
    step = 0
    ep = 0
    state = env.reset()
    mean_reward = 0
    is_over = False

    while not is_over:
        done = False
        
        while not done:
            #env.render()
            step += 1
            state = np.reshape(state, [1, size_in])
            state_buffer.append(state) ##상태 정보 stack
            
            pi = sess.run(L_out, feed_dict={STATE_IN:state})
            if(random.random() < epsilon):
                action = env.action_space.sample()
            else:
                action = np.argmax(pi)
                
            new_state, r, d, _ = env.step(action)
            state = new_state
            reward_buffer.append(r) ##보상 정보 stack
            
            if d:
                ep += 1
                epsilon = 1/(ep/20 + 1)
                
                eps = np.vstack(state_buffer)
                epr = np.vstack(reward_buffer)
                epr = discounted_reward(epr)##데이터 정리
                
                state_buffer, reward_buffer = [], []                
                sess.run(train, feed_dict={STATE_IN:eps, reward:epr})##학습
                mean_reward += step
                #################################학습 정보 쌓기
                
                if(ep % 5 == 0):
                    mean_reward = mean_reward/5
                    #epsilon = 1 / (mean_reward/100 + ep/100 + 2)
                    print('평균 : {}'.format(mean_reward))
                    
                    if(step >= 2000):
                        mean_reward = 0
                        saver.save(sess, save_file)
                        is_over = True
                        
                step = 0
                done = True
                state = env.reset()
env.close()

평균 : 40.4
평균 : 34.08
평균 : 53.215999999999994
평균 : 23.6432
평균 : 24.32864
평균 : 17.065728
평균 : 13.2131456
평균 : 16.642629120000002
평균 : 13.328525824000002
평균 : 14.8657051648
평균 : 11.97314103296
평균 : 59.19462820659201
평균 : 92.0389256413184
평균 : 94.40778512826368
평균 : 68.48155702565273
평균 : 22.896311405130547
평균 : 13.579262281026109
평균 : 30.915852456205222
평균 : 84.58317049124105
평균 : 205.91663409824824
평균 : 226.98332681964968
평균 : 184.59666536392993
평균 : 111.71933307278599
평균 : 37.7438666145572
평균 : 85.14877332291144
평균 : 269.0297546645823
평균 : 188.00595093291645
평균 : 259.4011901865833
평균 : 551.8802380373166
평균 : 2110.376047607463
